In [8]:
import requests
from bs4 import BeautifulSoup
import datetime
import sqlite3
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains

# Create or connect to SQLite database
conn = sqlite3.connect('jumia_products.db')
c = conn.cursor()

# Create table if it doesn't exist
c.execute('''
CREATE TABLE IF NOT EXISTS products (
    product_name TEXT,
    current_price TEXT,
    original_price TEXT,
    discount_percentage TEXT,
    rating TEXT,
    reviews TEXT
)
''')

# Base URL
base_url = "https://www.jumia.co.ke/home-office-appliances/?tag=JMALL&sort=rating"

# Initialize WebDriver
driver = webdriver.Chrome()  # or use the appropriate WebDriver

# Function to extract data from a page
def extract_data(page_source):
    soup = BeautifulSoup(page_source, "html.parser")
    products = soup.find_all('div', class_='info')
    
    for product in products:
        try:
            product_name = product.find('h3', class_='name').text.strip()
            current_price = product.find('div', class_='prc').text.strip()
            original_price = product.find('div', class_='old').text.strip() if product.find('div', class_='old') else None
            discount_percentage = product.find('div', class_='bdg _dsct _sm').text.strip() if product.find('div', class_='bdg _dsct _sm') else None
            rating = product.find('div', class_='stars _s').text.strip() if product.find('div', class_='stars _s') else None
            reviews = product.find('div', class_='rev').text.strip() if product.find('div', class_='rev') else None
            
            # Insert data into the database
            c.execute('''INSERT INTO products (date, product_name, current_price, original_price, discount_percentage, rating, reviews) 
                         VALUES (?, ?, ?, ?, ?, ?, ?)''',
                      (datetime.datetime.now(), product_name, current_price, original_price, discount_percentage, rating, reviews))
            conn.commit()
            
        except AttributeError:
            continue

# Function to scrape Jumia across all pages
def scrape_jumia():
    driver.get(base_url)
    time.sleep(5)  # Wait for the page to load
    
    while True:
        # Extract data from the current page
        extract_data(driver.page_source)
        
        try:
            # Find the 'Next' button using its class or link text and click it
            next_button = driver.find_element(By.CSS_SELECTOR, "a.pg[aria-label='Next Page']")
            ActionChains(driver).move_to_element(next_button).click().perform()
            
            # Wait for the next page to load
            time.sleep(5)
        except Exception as e:
            print(f"No more pages or error occurred: {e}")
            break

    # Close the WebDriver
    driver.quit()

# Start scraping
scrape_jumia()

# Select all from the table and print results
c.execute('''SELECT * FROM products''')
results = c.fetchall()
print(results)

# Close SQLite connection
conn.close()

No more pages or error occurred: Message: no such element: Unable to locate element: {"method":"css selector","selector":"a.pg[aria-label='Next Page']"}
  (Session info: chrome=127.0.6533.119); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
#0 0x562f61e946ca <unknown>
#1 0x562f61b65600 <unknown>
#2 0x562f61bb4bcb <unknown>
#3 0x562f61bb4eb1 <unknown>
#4 0x562f61bf8b24 <unknown>
#5 0x562f61bd78cd <unknown>
#6 0x562f61bf604a <unknown>
#7 0x562f61bd7643 <unknown>
#8 0x562f61ba7d31 <unknown>
#9 0x562f61ba879e <unknown>
#10 0x562f61e5c25b <unknown>
#11 0x562f61e601f2 <unknown>
#12 0x562f61e49615 <unknown>
#13 0x562f61e60d82 <unknown>
#14 0x562f61e2e25f <unknown>
#15 0x562f61e83e68 <unknown>
#16 0x562f61e84040 <unknown>
#17 0x562f61e9349c <unknown>
#18 0x7fc0f037d144 <unknown>

[('2024-08-27 22:35:47.335982', 'Ramtons RM/392-Pureit GKK 3000LT Germ Kit (1 YR WRTY)', 'KSh 3,360', 'KSh 